In [2]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt

In [1]:
15*12

180

In [3]:
# tables=tabula.read_pdf(f,
#                           lattice=True,
#                           pages="all",
#                           silent=True,
#                           multiple_tables=True,
#                           pandas_options={'header':None})
# df=pd.DataFrame()
# df=pd.concat([c for c in tables]).drop_duplicates()

In [51]:
f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\SBI\files\sbi01.pdf"
try:
    tables=tabula.read_pdf(f,
                              lattice=True,
                              pages="all",
                              silent=True,
                              multiple_tables=True,
                              pandas_options={'header':None})

    for k in [ c for c in tables if c.shape[1]>=6 and c.shape[0]>2]:
        k.drop(k.nunique(dropna=False)[(k.nunique(dropna=False) == 2)].index, axis=1,inplace=True)
        k.dropna(how='all',axis=1,inplace=True);   k.columns=list(range(k.shape[1]))

    df = pd.DataFrame()
    df = pd.concat([c for c in tables]).drop_duplicates()
    
except: pass

In [52]:
df=df.replace(r'^\s*$', np.nan, regex=True)
df.head()

,0,1,2,3,4,5,6,7
0,Txn\rDate,Value\rDate,Description,Ref\rNo./Cheque\rNo.,Branch\rCode,Debit,Credit,Balance
1,2 Jul\r2018,2 Jul\r2018,BY\rTRANSFER-\rNEFT*CITI00\r00003*CITIN\r18883...,TRANSFER\rFROM\r31994160443\r03 /,4430,NaN,"5,00,000.0\r0","5,03,809.59"
2,3 Jul\r2018,3 Jul\r2018,TO\rTRANSFER-\rINB CA to CC\r-,CIAABALXO6\rTRANSFER\rTO\r37171837744\rPERSONA...,99922,"5,00,000.0\r0",NaN,"3,809.59"
3,5 Jul\r2018,5 Jul\r2018,TO\rTRANSFER-\rINB\rBILL_BESCL\rPayments-,NSBI6455859\r854CH85374\r900\rTRANSFER\rTO\r31...,99922,"2,561.90",NaN,"1,247.69"
1,7 Jul\r2018,7 Jul\r2018,BY\rTRANSFER-\rINB\rIMPS818812\r200870/9731\r3...,MAC0001374\r62863\rMAC0001374\r62863\rTRANSFER...,99922,NaN,"60,000.00","61,247.69"


In [53]:
df = df[df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)

In [54]:
try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        print("\nSBI Column Headers Missing"); pass
    try:
        df = df.drop(["Init."], axis=1)
    except:
        try:
            df = df.drop(["Branch Name"], axis=1)
        except:pass

In [55]:
df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance|account statement').any(),axis=1)==True].index)]

In [56]:
df = pd.DataFrame(df.T.drop_duplicates().T)

In [57]:
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1, inplace=True)